In [1]:
import numpy as np
import pandas as pd
from PIL import Image
# import potrace
from collections import deque
from tqdm import tqdm


In [2]:
p = Image.open('Lekagul Roadways.bmp')
p_array = np.array(p)
p_norm = p_array / np.max(p_array)

gates = pd.read_csv("gate_loc.csv")
gates.r -= 1
gates.c -= 1
gates_dict = {}
for i in range(gates.shape[0]):
    gates_dict[gates.gates[i]] = (gates.r[i], gates.c[i])

hist = pd.read_csv("processed_dt.csv")
hist['path'] = hist['gate-name'] + '_' + hist['next_gate']
unique_edges = list(pd.unique(hist[~hist.next_gate.isnull()]['path']))

In [3]:
directions = [[1, 0], [1, -1], [1, 1], [0, -1], [0, 1], [-1, -1], [-1, 0], [-1, 1]]
edge_paths = {}
for path in tqdm(unique_edges):
    start, end = path.split('_')
    start_r, start_c = gates_dict[start]
    end_r, end_c = gates_dict[end]
    
    seen = set()
    seen.add(gates_dict[start])
    queue = deque([gates_dict[start]])
    not_found = True
    
    best_path = {}
    best_path[gates_dict[start]] = [(start_c, start_r)]
    
    while queue and not_found:
        r, c = queue.popleft()
        for dr, dc in directions:
            nr = r + dr
            nc = c + dc
            if (nr, nc) not in seen:
                seen.add((nr, nc))
                if nr == end_r and nc == end_c:
                    best_path[(nr, nc)]  = best_path[(r, c)].copy()
                    best_path[(nr, nc)].append((nc, nr))
                    not_found = False
                elif nr >= 0 and nr < 200 and nc >= 0 and nc < 200:
                    if p_norm[nr, nc] > 0.7:
                        queue.append((nr, nc))
                        if best_path.get((nr, nc)) is None:
                            best_path[(nr, nc)]  = best_path[(r, c)].copy()
                            best_path[(nr, nc)].append((nc, nr))
    edge_paths[path] = best_path[gates_dict[end]]

100%|██████████| 209/209 [00:02<00:00, 75.47it/s] 


In [8]:
" ".join([str(x) for x in edge_paths['general-gate1_entrance3'][0]])

'64 25'

In [40]:
def write_svg(edge_paths, nodes, file, size=200):
    size_scale = size / 200
    svg_header = '<svg width="' + str(size) + '" height="' + str(size) + '" xmlns="http://www.w3.org/2000/svg">'
    svg_body = []
    for edge in edge_paths.keys():
        svg_path = ['M ' + ' '.join([str(int(x * size_scale)) for x in edge_paths[edge][0]])]
        for l in edge_paths[edge][1:]:
            svg_path.append('L ' + ' '.join([str(int(x * size_scale)) for x in l]))
        svg_path_text = ' '.join(svg_path)
        svg_body.append(
            '<path d="' + svg_path_text + '" id="' \
            + edge + '" stroke-width="' + str(int(size_scale)) \
            + '" stroke="black" fill="transparent"/>'
        )
    for node in nodes.keys():
        svg_body.append(
            '<circle cx="' + str(int(nodes[node][1] * size_scale)) + 
            '" cy="' + str(int(nodes[node][0] * size_scale)) + 
            '" r="' + str(int(size_scale * 1.5)) + '" id="node-' 
            + node + '" fill="red"/>'
        )
    svg_content = svg_header + '\n' + '\n'.join(svg_body) + '\n' + '</svg>'
    with open(file, 'w') as f:
        f.writelines(svg_content)
    return

In [41]:
write_svg(edge_paths, gates_dict, 'test.svg', 600)

'<svg width="200" height="200200" xmlns="http://www.w3.org/2000/svg">\n<path d="M 64 25 L 63 26 L 63 27 L 63 28 L 63 29 L 64 30 L 64 31 L 64 32 L 65 33 L 66 34 L 66 35 L 67 36 L 67 37 L 68 38 L 68 39 L 69 40 L 69 41 L 70 42 L 70 43 L 71 44 L 71 45 L 72 46 L 73 47 L 74 48 L 74 49 L 75 50 L 76 51 L 76 52 L 77 53 L 77 54 L 78 55 L 78 56 L 79 57 L 80 58 L 81 59 L 81 60 L 81 61 L 82 62 L 83 63 L 84 64 L 85 65 L 85 66 L 86 67 L 86 68 L 86 69 L 86 70 L 86 71 L 85 72 L 85 73 L 86 74 L 86 75 L 87 76 L 88 77 L 88 78 L 89 79 L 90 80 L 91 81 L 91 82 L 91 83 L 91 84 L 91 85 L 90 86 L 90 87 L 90 88 L 89 89 L 90 90 L 91 90 L 92 91 L 92 92 L 93 93 L 93 94 L 93 95 L 93 96 L 93 97 L 93 98 L 93 99 L 93 100 L 94 101 L 94 102 L 94 103 L 95 104 L 95 105 L 96 106 L 96 107 L 97 108 L 97 109 L 98 110 L 99 111 L 99 112 L 100 113 L 101 114 L 102 115 L 103 116 L 103 117 L 104 118 L 105 119 L 106 120 L 106 121 L 107 122 L 107 123 L 108 124 L 108 125 L 109 126 L 110 127 L 111 128 L 112 129 L 112 130 L 113 131 L 114